In [29]:
import numpy as np
import pandas as pd
import copy
from collections import Counter

class TSP:
    def __init__(self, G):
        # データフレームの作成
        G = np.array(G).T # カラムごとにデータフレームに格納するので転置
        data = {key: value for (key,value) in zip (range(1, len(G)+1), G) }# 各カラムの設定
        G = pd.DataFrame(data, index = [i for i in range(1, len(data)+1)]) # グラフ構造
        P = {'G': G, 'r':[],'d':0, 'g':np.inf} # 部分問題　# r：確定経路に使われるエッジのリスト, d：確定経路の距離, g：下界値
        self.A = [P]  # 未解決部分問題の集合
        self.d_min = np.inf # 最短距離の暫定値
        self.second_g = np.inf # 部分問題の中で2番目に小さい下界値
        self.edge_list =[] # 最短経路に使われるエッジのリスト

    # 部分問題のグラフ構造の下界値(すでに確定している経路の距離を含まない)を求める関数
    def lower_bound(self, G):
        # 部分問題のグラフ構造の中で距離が最小となるエッジを取得
        G_sub = copy.copy(G) # G に影響を与えないため深いコピー
        d_min = np.inf
        for idx in G_sub.index:
            if G_sub.loc[idx].min() < d_min:
                d_min = G_sub.loc[idx].min()
                idx_row = idx
                idx_column = int(G_sub.loc[idx].argmin())
                
        # 各行の最小値のカラムを追加後， 各行から引く
        G_sub['min_row'] = G_sub.min(axis = 1) 
        for idx in G_sub.index:
            G_sub.loc[idx][:-1] -= G_sub['min_row'][idx]

        # 各列の最小値の行を追加後，　各列から引く．
        min_column = pd.DataFrame(G_sub.min(axis = 0)[:-1], columns = ['min_column']).T
        G_sub = G_sub.append(min_column)
        G_sub.iloc[:-1, :-1] -= G_sub.loc['min_column'][:-1]

        # 下界値(すでに確定していた経路の距離は含まない)
        lower_bound = G_sub['min_row'].sum() + G_sub.loc['min_column'].sum()
        return lower_bound

    # 部分問題が与えられた時， 距離が最短のエッジを通るかどうかで部分問題に分割する関数．
    def split_(self, P):
        G = P['G'] # グラフ構造
        r = P['r'] # 確定経路
        d = P['d'] #  確定経路の距離

        # カラム数が1の場合は終端判定を行う
        if len(G) == 1:
            # 確定経路，距離を更新
            d += G.min().min()
            r.append( [G.index[0], G.columns[0]])
            
            # 確定経路のエッジの出現回数を数える
            r_flat = np.array(r).reshape(-1,)
            counter = Counter(r_flat)
            
            # 終端判定
            if counter.most_common()[:-1][0][1] < 2: # エッジの出現回数の最小値が2未満=>巡回していない
                return True # 探索を続行
            elif self.d_min < d : # 最短距離が暫定解よりも大きい
                return True
            elif self.second_g < d: # 最短距離が未解決部分問題の下界値の最小値よりも大きい
                self.d_min = d
                return True
            else: # 探索終了
                self.d_min = d
                self.edge_list = r
                return False
        
        # カラム数が2以上の場合， 部分問題に分ける．
        else:
            # 距離が最小のエッジを取得
            d_min = np.inf
            for idx in G.index:
                if G.loc[idx].min() < d_min:
                    d_min = G.loc[idx].min()
                    idx_row = idx
                    idx_column = int(G.loc[idx].argmin())

            # 最短エッジを通らない部分問題
            G1 =  copy.copy(G)
            G1.loc[idx_row][idx_column] = np.inf # 最短エッジは通らないので無限大に．
            r1 = copy.copy(r)
            d1 = copy.copy(d)
            g1 =  self.lower_bound(G1) + d1# 下界値(確定経路の距離を含む)
            P1 = {'G':G1,'r':r1,  'd': d1,'g':g1} # 部分問題1

            # 最短エッジを通る部分問題
            G2 = G.drop(idx_row).drop(idx_column, axis=1)
            r2 = copy.copy(r)
            r2.append([idx_row, idx_column]) # 確定経路の更新
            d2 = int(d + d_min) # 確定距離の更新
            if idx_row in G2.index and idx_row in G2.column:
                G2.loc[idx_column][idx_row]  = np.inf # 最短エッジを逆向きに通るのは非合理的なので対角成分を∞に
            g2 = self.lower_bound(G2)+d2
            P2 = {'G':G2, 'r':r2,'d': d2, 'g':g2} # 部分問題1
            return P1, P2
    
    # 解を求める関数
    def main(self):
        # 下界値が最小の部分問題(A[0])を探索し， 新たな部分問題 or Trueが返される限り探索続行．
        while self.split_(self.A[0]):
            try: # A[0]のカラムが2以上の場合， 新たな部分問題2つが返される．
                P1, P2 = self.split_(self.A[0])
            except: #A[0]のカラム数が1かつ終端条件を満たさない場合
                self.A.pop(0) # A[0] を消去．
                P1, P2 = self.split_(self.A[0]) # 新たな部分問題を取得． 
            self.A.pop(0) # 先頭の部分問題を消去
            self.A.append(P2) # 新たな部分問題を追加
            self.A.append(P1)
            self.A =  sorted(self.A, key=lambda x:x['g']) # 下界値が小さい順にソート
            self.second_g = self.A[1]['g'] # 2番目に小さい下界値を更新

        # 最短経路の文字列の作成
        r = self.edge_list # 最短経路のエッジのリスト 
        route_list = [] # 最短経路のノード(通る順番に並び替える)
        route_list.append(r[0][0]) # 最初に通るノード
        next_ = r[0][1] # 次に通るノード
        r.pop(0) # 先頭のエッジを消去
        while len(r) > 0:
            for i in range(len(r)): 
                if r[i][0] == next_: # 次に通るエッジを探索
                    route_list.append(r[i][0]) 
                    next_ =r[i][1]
                    r.pop(i)
                    break
        shortest_route = str(route_list[0]) # 最短経路(文字列)
        for i in range(1, len(route_list)):
            shortest_route  += " → " + str(route_list[i])       
        shortest_route += " → " + str(route_list[0])
        
        return shortest_route , self.d_min

# 実行
G = [
    [np.inf, 21, 7, 13, 15],
    [11, np.inf, 19, 12, 25],
    [15, 24, np.inf, 13, 5], 
    [6, 17, 9, np.inf, 22],
    [28, 6, 11, 5, np.inf]
]
q = TSP(G)
r, z = q.main()
print('最短経路：',r)
print('最短距離:', int(z))

最短経路： 3 → 5 → 2 → 4 → 1 → 3
最短距離: 36
